# DATA2001 Group Assignment: Quantifying Resource Availability in Greater Sydney SA2 Regions

## 1. Setup & Dependencies

### 1.1 Imports

In [94]:
# Standard library
import json

# Data manipulation
import numpy as np
import pandas as pd
import geopandas as gpd

# Database connectivity
from sqlalchemy import create_engine, text
from geoalchemy2 import Geometry, WKTElement

# Spatial geometry
from shapely.geometry import MultiPolygon, Point, Polygon

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import contextily as ctx
import folium
from folium.plugins import MarkerCluster, FeatureGroupSubGroup

# Statistical analysis
from scipy.stats import pearsonr, spearmanr

### 1.2 PostgreSQL Connector Setup
To streamline database interactions, we define a pgconnect() function that reads connection details from Credentials.json and creates a SQLAlchemy engine pointing to our Postgres instance. This centralizes all authentication and configuration in one place. We then use a query() helper that takes an open connection and a SQL string, executes the statement, and returns the results directly as a pandas DataFrame—making it trivial to pull data from the database into our analysis pipeline.

In [97]:
credentials = "Credentials.json"
def pgconnect(credential_filepath, db_schema="public"):
    """
    Reads credentials.json, prefers the 'database' key, then 'dbname',
    then falls back to the user. Returns (engine, connection).
    """
    with open(credential_filepath, "r") as f:
        creds = json.load(f)

    host   = creds["host"]
    port   = creds["port"]
    user   = creds["user"]
    pw     = creds["password"]
    dbname = creds.get("database") or creds.get("dbname") or user

    url = f"postgresql+psycopg2://{user}:{pw}@{host}:{port}/{dbname}"

    try:
        engine = create_engine(url, echo=False)
        conn   = engine.connect()
        print(f"Connected successfully to '{dbname}'.")
    except Exception as e:
        print("Unable to connect to the database:")
        print(e)
        engine, conn = None, None

    return engine, conn

def query(engine, sqlcmd, args=None, df=True):
    """
    Convenience wrapper around pandas.read_sql_query or raw execute.
    - If df=True: returns a DataFrame via pandas.read_sql_query()
    - Else: runs a raw execute() on engine and returns fetched results.
    """
    try:
        if df:
            return pd.read_sql_query(sqlcmd, engine, params=args)
        else:
            result = engine.execute(text(sqlcmd), args).fetchall()
            return result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered:", e)
        return None


### 1.3 Establish Database Connection
This line invokes our connector and prepares the session for all subsequent database work:

In [100]:
db, conn = pgconnect(credentials)

Unable to connect to the database:
(psycopg2.OperationalError) connection to server at "localhost" (::1), port 5432 failed: FATAL:  database "localhost" does not exist

(Background on this error at: https://sqlalche.me/e/20/e3q8)


### 1.4 Verify PostGIS Installation
Before running any spatial operations, we perform a quick sanity check by calling the PostGIS_Version() function through our query() helper. A returned version string confirms that the PostGIS extension is installed and that spatial SQL functions are available in the database.

In [103]:
query(conn, "select PostGIS_Version()")

Error encountered: 'NoneType' object has no attribute 'cursor'


## 2. Data Ingestion 

### 2.1 Import Core CSV Tables
Loads business, income, and population data from their respective CSV files. The raw data is initially previewed for structural verification, then deduplicated, and finally loaded into the public.businesses, public.population, and public.income tables in PostgreSQL. This step prepares these foundational datasets for analytical use and linkage with other data.

In [ ]:
business = pd.read_csv("data/Businesses.csv")
print("Businesses.csv head:")
print(business.head())

income = pd.read_csv("data/income.csv")
print("\nincome.csv head:")
print(income.head())

population = pd.read_csv("data/Population.csv")
print("\nPopulation.csv head:")
print(population.head())

print("\nLoading CSVs into PostgreSQL tables...")
for csv_file, table_name in [
    ("data/Businesses.csv", "businesses"),
    ("data/Population.csv", "population"),
    ("data/Income.csv", "income"),
]:
    df = pd.read_csv(csv_file)
    df.drop_duplicates(inplace=True)
    df.to_sql(
        table_name,
        conn,
        schema = "public",
        if_exists = "replace",
        index = False,
        method = "multi")
    print(f"Loaded {table_name}: {len(df)} rows")

### 2.2 Create Core Spatial Tables
Establishes the initial PostGIS schema for SA2 boundaries and transport stops by creating (or recreating) the public.sa2_regions and public.stops tables. These tables are defined with appropriate geometry columns (MULTIPOLYGON for regions, POINT for stops) and GIST indexes to ensure efficient spatial querying. The sa2_regions table created here serves as a preliminary structure that will be further refined.

In [ ]:
create_sa2_sql = """
DROP TABLE IF EXISTS public.sa2_regions;
CREATE TABLE public.sa2_regions (
    sa2_code_2021 TEXT PRIMARY KEY,
    sa2_name_2021 TEXT,
    geometry      GEOMETRY(MULTIPOLYGON,4326)
);
CREATE INDEX IF NOT EXISTS sa2_regions_geom_idx
    ON public.sa2_regions USING GIST (geometry);

DROP TABLE IF EXISTS public.stops;
CREATE TABLE public.stops (
    stop_id   TEXT PRIMARY KEY,
    stop_name TEXT,
    geometry  GEOMETRY(POINT,4326)
);
CREATE INDEX IF NOT EXISTS stops_geom_idx
    ON public.stops USING GIST (geometry);
"""

conn.execute(text(create_sa2_sql))
print("\nCore spatial tables (sa2_regions, stops) structure created/recreated.")

### 2.3 Load SA2 Boundary Shapefile into PostGIS
Loads SA2 geographic boundaries into the public.sa2_regions PostGIS table. Data is read from an SA2 shapefile, processed to standardize column names (including sa2_code, sa2_name, sa4_name_2021, gcc_name_2021), ensure the CRS is EPSG:4326, and remove duplicates. This processed data then replaces any prior version of the public.sa2_regions table. Finally, a primary key on sa2_code and a GIST spatial index on the geometry are conditionally created to ensure data integrity and optimize spatial analysis.

In [ ]:
sa2 = (
    gpd.read_file("data/SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp")[
        ["SA2_CODE21", "SA2_NAME21", "SA4_NAME21", "GCC_NAME21", "geometry"]
    ]
    .rename(
        columns={
            "SA2_CODE21": "sa2_code",
            "SA2_NAME21": "sa2_name",
            "SA4_NAME21": "sa4_name_2021",
            "GCC_NAME21": "gcc_name_2021",
        }
    )
    .to_crs(4326)
    .drop_duplicates()
)

sa2.to_postgis(
    "sa2_regions", conn,
    schema="public",
    if_exists="replace",
    index=False,
    dtype={"geometry": Geometry("MULTIPOLYGON", 4326)}
)
print(f"\nInserted {len(sa2):,} SA2 rows, replacing sa2_regions table.")

conn.execute(text("""
DO $$
BEGIN
  IF NOT EXISTS (
      SELECT 1 FROM pg_constraint
      WHERE conrelid = 'public.sa2_regions'::regclass
        AND conname = 'sa2_regions_pkey'
  ) THEN
      ALTER TABLE public.sa2_regions
        ADD CONSTRAINT sa2_regions_pkey PRIMARY KEY (sa2_code);
  END IF;

  IF NOT EXISTS (
      SELECT 1 FROM pg_indexes
      WHERE schemaname = 'public'
        AND tablename  = 'sa2_regions'
        AND indexname  = 'sa2_regions_geom_idx'
  ) THEN
      CREATE INDEX sa2_regions_geom_idx
        ON public.sa2_regions USING GIST (geometry);
  END IF;
END $$;
"""))
print("Primary key and GIST index on sa2_regions verified/created.")

### 2.4 Insert Transport Stops from GTFS
Populates the public.stops table with public transport stop locations. Data sourced from Stops.txt (typically part of a GTFS feed) is deduplicated and transformed into a spatial format by creating POINT geometries from latitude/longitude coordinates (CRS EPSG:4326). These stop locations are then appended to the public.stops table, leveraging the schema and GIST index established earlier, to provide a spatial layer for transit infrastructure analysis.

In [ ]:
stops = pd.read_csv("data/Stops.txt")
stops.drop_duplicates(inplace=True)

stops_gdf = gpd.GeoDataFrame(
    stops[["stop_id","stop_name"]],
    geometry=gpd.points_from_xy(stops.stop_lon, stops.stop_lat),
    crs=4326
)

stops_gdf.to_postgis("stops", conn,
                     schema="public",
                     if_exists="append",
                     index=False,
                     dtype={"geometry": Geometry("POINT",4326)})

print(f"\nInserted {len(stops_gdf):,} stops into public.stops table.")

### 2.5 Load School Catchment Polygons
Integrates school catchment boundaries into the public.schools PostGIS table. Data from separate shapefiles for primary, secondary, and future catchments is read, standardized (CRS to EPSG:4326, columns renamed, 'sector' attribute added), consolidated, and deduplicated. This comprehensive catchment data is then loaded into a newly created public.schools table, structured with a surrogate primary key (gid), MULTIPOLYGON geometry, and a GIST spatial index to facilitate analysis of educational access areas.

In [ ]:
def read_catch(path):
    gdf = gpd.read_file(path).to_crs(4326)
    gdf = gdf.rename(columns={
        "USE_ID": "school_id",
        "CATCH_TYPE": "stage"
    })
    gdf["sector"] = "Government"
    return gdf[["school_id", "sector", "stage", "geometry"]]

gdf_primary   = read_catch("data/catchments/catchments_primary.shp")
gdf_secondary = read_catch("data/catchments/catchments_secondary.shp")
gdf_future    = read_catch("data/catchments/catchments_future.shp")

schools = pd.concat([gdf_primary, gdf_secondary, gdf_future], ignore_index=True)
schools.drop_duplicates(inplace=True)
print(f"\nTotal unique school polygons prepared: {len(schools):,}")

conn.execute(text("""
DROP TABLE IF EXISTS public.schools;
CREATE TABLE public.schools (
    gid       SERIAL PRIMARY KEY,
    school_id TEXT,
    sector    TEXT,
    stage     TEXT,
    geometry  GEOMETRY(MULTIPOLYGON,4326)
);
CREATE INDEX schools_geom_idx
  ON public.schools USING GIST (geometry);
"""))
print("Recreated public.schools table with GIST index.")

schools.to_postgis("schools", conn, schema="public",
                   if_exists="append", index=False,
                   dtype={"geometry": Geometry("MULTIPOLYGON",4326)})
print("School catchments loaded into public.schools table.")

### 2.6 Build Indexes on SA2 Key Columns
Optimizes data retrieval performance by creating B-tree indexes on the SA2 key columns (e.g., sa2_code or sa2_code21) within the public.businesses, public.population, and public.income tables. These indexes are created if they don't already exist and significantly accelerate join operations between these attribute tables and the public.sa2_regions spatial table, enabling efficient linkage of demographic and economic data to geographic areas.

In [ ]:
conn.execute(text("""
    CREATE INDEX IF NOT EXISTS businesses_sa2_idx
      ON public.businesses (sa2_code);
    CREATE INDEX IF NOT EXISTS population_sa2_idx
      ON public.population (sa2_code);
    CREATE INDEX IF NOT EXISTS income_sa2_idx
      ON public.income (sa2_code21);
"""))
print("\nIndexes created on sa2_code columns for businesses, population, and income tables.")

### 2.7 Validate Table Loads
Confirms the completeness of the data ingestion process by querying and displaying the row counts for all key tables created or populated in the public schema: sa2_regions, stops, schools, businesses, population, and income. This serves as a fundamental validation step to ensure that each dataset has been loaded as expected.

In [ ]:
print("\nRow counts in public tables:")
row_counts_df = pd.read_sql("""
SELECT 'sa2_regions' AS table_name, COUNT(*) AS rows FROM public.sa2_regions
UNION ALL
SELECT 'stops',        COUNT(*) FROM public.stops
UNION ALL
SELECT 'schools',      COUNT(*) FROM public.schools
UNION ALL
SELECT 'businesses',   COUNT(*) FROM public.businesses
UNION ALL
SELECT 'population',   COUNT(*) FROM public.population
UNION ALL
SELECT 'income',       COUNT(*) FROM public.income;
""", conn)
print(row_counts_df)

### 2.8 Confirm Spatial Metadata
Verifies the consistency of spatial data across the database by querying the geometry_columns system catalog. This lists all tables in the public schema that contain geometry columns, along with their Spatial Reference ID (SRID). This check ensures that all spatial data correctly uses the designated coordinate system (e.g., EPSG:4326), which is crucial for accurate spatial operations and analysis.

In [ ]:
print("\nSRID of geometry columns in public schema:")
srid_df = pd.read_sql("""
SELECT f_table_schema AS schema_name, f_table_name AS table_name, f_geometry_column AS geometry_column, srid
FROM   geometry_columns
WHERE  f_table_schema = 'public';
""", conn)
print(srid_df)

### 2.9 Connection Diagnostics
Provides diagnostic information about the current database connection by displaying the SQLAlchemy engine URL, the DSN (Data Source Name) parameters of the connection, and the name of the currently active database (queried directly). This output helps in verifying that the script is connected to the intended database environment and can aid in troubleshooting any connection-related issues.

In [ ]:
if 'engine' in locals() or 'engine' in globals():
    print("\nengine    →", engine.url)
else:
    print("\nEngine object 'engine' not found for diagnostics.")

if 'conn' in locals() and conn and conn.connection:
    try:
        dsn_params = conn.connection.get_dsn_parameters()
        print("conn DSN  →", dsn_params)
        current_db_df = pd.read_sql(text("SELECT current_database()"), conn) # conn instead of db
        print("current_db→", current_db_df.iloc[0,0])
    except AttributeError:
        print("Could not retrieve DSN parameters or current_database using this connection object.")
    except Exception as e:
        print(f"An error occurred during connection diagnostics: {e}")
else:
    print("Connection object 'conn' not available or not active for diagnostics.")

### 2.10 WEB API WILSON

this code queries the NSW SIX Maps POI API to retrieve POIs within specified bounding box defined by minimum and maximum longitude and latitude coordinates. It sends an HTTP GET request with spatial parameters, parsed the returned JSON response, and then extracts relevant POI attributes and coordinates. The result is returned as a Pandas DataFrame containing POI name, type, group, labels, and geographic location, along with the raw JSON response for optional inspection

In [ ]:
import requests
import pandas as pd

def get_pois_from_api(minx, miny, maxx, maxy):
    
    url = "https://maps.six.nsw.gov.au/arcgis/rest/services/public/NSW_POI/MapServer/0/query"
    
    params = {
        "where": "1=1",
        "geometry": f"{minx},{miny},{maxx},{maxy}",
        "geometryType": "esriGeometryEnvelope",
        "spatialRel": "esriSpatialRelIntersects",
        "outFields": "*",
        "returnGeometry": "true",
        "f": "json"
    }
    
    response = requests.get(url, params=params)

    if response.status_code != 200:
        print("Error fetching data:", response.status_code)
        return pd.DataFrame(), {}
    
    result = response.json()

    if "features" not in result:
        return pd.DataFrame(), result
    
    pois = []
    
    for feature in result["features"]:
        attr = feature["attributes"]
        geom = feature.get("geometry", {})
        
        pois.append({
            "POIName": attr.get("poiname", None),
            "POIType": attr.get("poitype", None),
            "POIGroup": attr.get("poigroup", None),
            "POILabel": attr.get("poilabel", None),
            "POIAltLabel": attr.get("poialtlabel", None),
            "POILabelType": attr.get("poilabeltype", None),
            "Longitude": geom.get("x", None),
            "Latitude": geom.get("y", None)
        })
    
    return pd.DataFrame(pois), result


This code retreives POIs for SA2 regions within our selected SA4 areas (Sydney - Blacktown, Parramatta, and Ryde) by querying their bounding boxes from a PostgreSQL. It parses the bounding box coordinates, calls the provided API to fetch POIs for each region, appends SA2 metadata to the results, and pauses between requests for 1s to not hit API limits. The collected POIs are then combined and stored in the nsw_poi table within the database, replacing any existing data

In [ ]:
import time
from sqlalchemy import text, String, Integer, Float, DateTime

selected_sa4 = ["Sydney - Blacktown", "Sydney - Parramatta", "Sydney - Ryde"]
selected_sa4_str = ", ".join(f"'{sa4}'" for sa4 in selected_sa4)

sa2_df = pd.read_sql(f"""
SELECT sa2_code, sa2_name, sa4_name_2021, ST_Extent(geometry) AS bbox
FROM public.sa2_regions
WHERE sa4_name_2021 IN ({selected_sa4_str})
GROUP BY sa2_code, sa2_name, sa4_name_2021
""", conn)

def bbox_parse(bbox_str):
    bbox_str = bbox_str.replace("BOX(", "").replace(")", "")
    min_point, max_point = bbox_str.split(",")
    minx, miny = map(float, min_point.strip().split(" "))
    maxx, maxy = map(float, max_point.strip().split(" "))
    return minx, miny, maxx, maxy

sa2_df[["min_lon", "min_lat", "max_lon", "max_lat"]] = sa2_df["bbox"].apply(lambda x: pd.Series(bbox_parse(x)))

results = []

for index, row in sa2_df.iterrows():
    sa2_code = row["sa2_code"]
    sa2_name = row["sa2_name"]
    min_lon = row["min_lon"]
    min_lat = row["min_lat"]
    max_lon = row["max_lon"]
    max_lat = row["max_lat"]

    print(f"Fetching POIs for SA2: {sa2_name}")

    poi_df, result_p = get_pois_from_api(min_lon, min_lat, max_lon, max_lat)
    # print(result_p)

    if poi_df.empty:
        continue

    poi_df["SA2_CODE"] = sa2_code
    poi_df["SA2_NAME"] = sa2_name
    results.append(poi_df)

    time.sleep(1)

print("Finished fetching POIs.")

if results:
    final_poi_df = pd.concat(results, ignore_index=True)
    print(f"Total POIs fetched: {len(final_poi_df)}")
    display(final_poi_df.head())
    

    final_poi_df.to_sql("nsw_poi", db, if_exists="replace", index=False, dtype={
        "POIName": String(),
        "POIType": String(),
        "POIGroup": Integer(),
        "POILabel": String(),
        "POIAltLabel": String(),
        "POILabelType": String(),
        "Longitude": Float(),
        "Latitude": Float(),
        "SA2_CODE": String(),
        "SA2_NAME": String(),
        "timestamp": String()   
    })

    print("POIs successfully inserted into table 'nsw_poi'.")
else:
    print("No POIs found.")

## 3. Score Calculation

### 3.1 Business z-Score 

We focus on industries that directly provide everyday goods, services, and amenities to local residents. To that end, we exclude:

- **Heavy-industry & utilities**  
  _(Mining, Manufacturing, Agriculture, Electricity/Gas/Water/Waste, Construction)_  
  These sectors typically locate outside dense population centers and don’t reflect the local availability of shops, clinics, or leisure services.

- **Public Administration & Safety**  
  Although essential, these are government-run functions rather than private-sector businesses, and so are better handled by dedicated public-sector layers.

- **Administrative & Support Services**  
  Roles like cleaning, security, or back-office operations don’t signal consumer-facing amenities; they rarely indicate local resource richness from a resident’s perspective.

In [ ]:
sql = text("""
DROP TABLE IF EXISTS public.sa2_summary;

CREATE TABLE public.sa2_summary AS
SELECT
  sr.sa2_code,
  sr.sa2_name,
  SUM(b.total_businesses)
    / (pop.total_people   / 1000.0) AS business_count
FROM businesses b
  JOIN sa2_regions    sr  USING (sa2_name)
  JOIN population pop USING (sa2_name)
WHERE sr.sa4_name_2021 IN (
    'Sydney - Blacktown',
    'Sydney - Parramatta',
    'Sydney - Ryde'
  )
  AND b.industry_name IN (
    'Health Care and Social Assistance',
    'Education and Training',
    'Retail Trade',
    'Accommodation and Food Services',
    'Transport, Postal and Warehousing',
    'Financial and Insurance Services',
    'Professional, Scientific and Technical Services',
    'Arts and Recreation Services',
    'Rental, Hiring and Real Estate Services'
  )
  AND total_people >= 100
GROUP BY
  sr.sa2_code,
  sr.sa2_name,
  pop.total_people
""")

conn.execute(sql)


summary_df = query(conn, "SELECT * FROM public.sa2_summary;")
print(summary_df.head())

### 3.2 Transport Stops z-Score  

We quantify local transit access by counting every bus, train and tram stop located within each SA2 boundary. A higher `stops_count` not only signals more frequent service options, but also reflects greater ease of movement—letting residents run errands, commute or access leisure activities without relying on a car. By mapping stop density in this way, we capture how well-connected each neighbourhood is to the wider city network.

In [ ]:
sql = text("""
-- 1) Add the new column if it doesn't already exist
ALTER TABLE public.sa2_summary
ADD COLUMN IF NOT EXISTS stops_count INTEGER;

-- 2) Update that column by counting stops per SA2
UPDATE public.sa2_summary AS summ
SET stops_count = sub.stops_count
FROM (
  SELECT
    sr.sa2_name,
    COUNT(*) AS stops_count
  FROM sa2_regions AS sr
  JOIN stops AS s
    ON ST_Contains(sr.geometry, s.geometry)
  WHERE
    sr.sa4_name_2021 IN (
      'Sydney - Blacktown',
      'Sydney - Parramatta',
      'Sydney - Ryde'
    )
  GROUP BY
    sr.sa2_name
) AS sub
WHERE summ.sa2_name = sub.sa2_name;
""")

conn.execute(sql)

summary_df = query(
    conn,
    "SELECT * FROM public.sa2_summary;"
)
print(summary_df.head())

### 3.3 Schools z-Score

We gauge local education availability by counting the number of unique school-catchment polygons that overlap each SA2, then normalizing this as **catchments per 1 000 young residents**. Higher values indicate more schooling options relative to the size of the youth population.

- We derive **young_pop** by summing age bands 0–4, 5–9, 10–14 and 15–19 for each SA2.  
- We exclude any SA2 with **young_pop < 100** to avoid unstable per-capita rates in areas with very few children.  
- The raw catchment count is divided by **(young_pop / 1 000)**, ensuring that areas with smaller youth populations don’t produce disproportionately large metrics.  

In [ ]:
sql = text("""
-- 1) Add the new column if it doesn't already exist
ALTER TABLE public.sa2_summary
ADD COLUMN IF NOT EXISTS catchments_count DOUBLE PRECISION;

-- 2) Update that column by computing catchments per 1 000 young people
WITH pop_young AS (
  SELECT
    sa2_name,
    (
      "0-4_people"
    + "5-9_people"
    + "10-14_people"
    + "15-19_people"
    ) AS young_pop
  FROM population
)
UPDATE public.sa2_summary AS summ
SET catchments_count = sub.catchments_count
FROM (
  SELECT
    sr.sa2_name,
    COUNT(*)::float
      / (pyoung.young_pop / 1000.0) AS catchments_count
  FROM schools      AS s
  JOIN sa2_regions AS sr
    ON  ST_Intersects(sr.geometry, s.geometry)
    AND NOT ST_Touches(   sr.geometry, s.geometry)
  JOIN pop_young   AS pyoung
    ON pyoung.sa2_name = sr.sa2_name
  WHERE
    sr.sa4_name_2021 IN (
      'Sydney - Blacktown',
      'Sydney - Parramatta',
      'Sydney - Ryde'
    )
    AND pyoung.young_pop >= 100
  GROUP BY
    sr.sa2_name,
    pyoung.young_pop
) AS sub
WHERE summ.sa2_name = sub.sa2_name;
""")

conn.execute(sql)

summary_df = query(
    conn,
    "SELECT * FROM public.sa2_summary;"
)
print(summary_df.head())

### 3.4 Points of Interest z-Score 

We measure local amenity richness by counting only those POIs that represent everyday, community-facing services within each SA2. All other facility types are excluded to ensure the metric reflects true neighborhood resources:

- **Industrial & Utility Sites**  
  (e.g., Filtration Plant, Sewage Works, Pumping Station)  
  These support core infrastructure but are not public amenities.

- **Large-Scale Commercial & Tourism Facilities**  
  (e.g., Tourist Park, Silo – Commercial, Quarry – Open Cut)  
  Typically located outside residential centers and not part of everyday life.

- **Broad or Redundant Place Categories**  
  (e.g., City, Suburb, Locality)  
  Geographic labels rather than specific service points.

- **Niche or Infrequent-Use Sites**  
  (e.g., Motor Racing Track, Helipad, Gaol)  
  Specialized venues with limited regular community access.

In [ ]:
sql = text("""
-- 1) Add the new column if it doesn't already exist
ALTER TABLE public.sa2_summary
ADD COLUMN IF NOT EXISTS poi_count INTEGER;

-- 2) Update that column by counting POIs per SA2
UPDATE public.sa2_summary AS summ
SET poi_count = sub.poi_count
FROM (
  SELECT
    sr.sa2_name,
    COUNT(*) AS poi_count
  FROM nsw_poi AS n
  JOIN sa2_regions AS sr
    ON sr.sa2_name = n."SA2_NAME"
  WHERE
    sr.sa4_name_2021 IN (
      'Sydney - Blacktown',
      'Sydney - Parramatta',
      'Sydney - Ryde'
    )
    AND n."POIType" IN (
      'Library',
      'Park',
      'Community Facility',
      'Child Care Centre',
      'Primary School',
      'High School',
      'General Hospital',
      'Community Medical Centre',
      'Transport Interchange',
      'Post Office',
      'Sports Centre / Sports Field',
      'Art Gallery / Museum'
    )
  GROUP BY
    sr.sa2_name
) AS sub
WHERE summ.sa2_name = sub.sa2_name;
""")

conn.execute(sql)

summary_df = query(
    conn,
    "SELECT * FROM public.sa2_summary;"
)
print(summary_df.head())

### 3.5 Composite Score & Sigmoid Transformation

We standardize and merge our four resource metrics—`business_count`, `stops_count`, `catchments_count`, and `poi_count`—into a single, comparable score:

- **Z-Score Standardization**  
  Convert each raw count into a z-score by subtracting the overall mean and dividing by the population standard deviation. This puts all metrics on the same scale, so large-count categories don’t dominate the index.

- **Summation of Standardized Metrics**  
  Add the four z‐scores together to create `z_total`. Positive totals indicate above-average resource levels across all categories; negative totals indicate below-average.

- **Sigmoid Transformation**  
  Apply a logistic (sigmoid) function to `z_total` to squash the result into the (0, 1) range. A final `well_resourced_score` near 1 means very well-resourced; near 0 means poorly-resourced.

In [ ]:
sql = text("""
ALTER TABLE public.sa2_summary
  ADD COLUMN IF NOT EXISTS z_business          DOUBLE PRECISION,
  ADD COLUMN IF NOT EXISTS z_stops             DOUBLE PRECISION,
  ADD COLUMN IF NOT EXISTS z_schools           DOUBLE PRECISION,
  ADD COLUMN IF NOT EXISTS z_poi               DOUBLE PRECISION,
  ADD COLUMN IF NOT EXISTS z_total             DOUBLE PRECISION,
  ADD COLUMN IF NOT EXISTS well_resourced_score DOUBLE PRECISION;

WITH stats AS (
  SELECT
    AVG(business_count)      AS mean_bc,
    STDDEV_POP(business_count) AS sd_bc,
    AVG(stops_count)         AS mean_sc,
    STDDEV_POP(stops_count)    AS sd_sc,
    AVG(catchments_count)    AS mean_cc,
    STDDEV_POP(catchments_count) AS sd_cc,
    AVG(poi_count)           AS mean_pc,
    STDDEV_POP(poi_count)      AS sd_pc
  FROM public.sa2_summary
)
UPDATE public.sa2_summary
SET
  z_business = (COALESCE(business_count, 0) - stats.mean_bc) / NULLIF(stats.sd_bc,0),
  z_stops    = (COALESCE(stops_count, 0)    - stats.mean_sc) / NULLIF(stats.sd_sc,0),
  z_schools  = (COALESCE(catchments_count, 0) - stats.mean_cc) / NULLIF(stats.sd_cc,0),
  z_poi      = (COALESCE(poi_count, 0)      - stats.mean_pc) / NULLIF(stats.sd_pc,0),
  z_total    = (
      (COALESCE(business_count, 0)   - stats.mean_bc) / NULLIF(stats.sd_bc,0)
    + (COALESCE(stops_count, 0)      - stats.mean_sc) / NULLIF(stats.sd_sc,0)
    + (COALESCE(catchments_count, 0) - stats.mean_cc) / NULLIF(stats.sd_cc,0)
    + (COALESCE(poi_count, 0)        - stats.mean_pc) / NULLIF(stats.sd_pc,0)
  ),
  well_resourced_score = 1.0 / (1.0 + EXP(-(
      (business_count   - stats.mean_bc) / NULLIF(stats.sd_bc,0)
    + (stops_count      - stats.mean_sc) / NULLIF(stats.sd_sc,0)
    + (catchments_count - stats.mean_cc) / NULLIF(stats.sd_cc,0)
    + (poi_count        - stats.mean_pc) / NULLIF(stats.sd_pc,0)
  )))
FROM stats;
""")

conn.execute(sql)

summary_df = query(
    conn,
    """
    SELECT 
      sa2_code, 
      sa2_name, 
      z_business, 
      z_stops, 
      z_schools, 
      z_poi, 
      z_total, 
      well_resourced_score
    FROM public.sa2_summary
    """
)
print(summary_df.head())

## 4. Score Analysis

### 4.1 Component Distributions

Visualizes the contribution of individual components (businesses, transit stops, schools, points of interest) to the overall "well-resourced score" for each SA2 region. This analysis uses boxplots to show the distribution of z-scores for each component, highlighting the variability and typical impact of each factor on the final scores.

In [ ]:
components_df = pd.read_sql(text("""
    SELECT
        sa2_name,
        z_business,
        z_stops,
        z_schools,
        z_poi,
        well_resourced_score
    FROM public.sa2_summary
    ORDER BY well_resourced_score DESC
"""), conn)

melted_df = pd.melt(components_df,
                    id_vars=['sa2_name', 'well_resourced_score'],
                    value_vars=['z_business', 'z_stops', 'z_schools', 'z_poi'],
                    var_name='Component',
                    value_name='Z-Score')

component_names = {
    'z_business': 'Businesses',
    'z_stops': 'Transit Stops',
    'z_schools': 'Schools',
    'z_poi': 'Points of Interest'
}
melted_df['Component'] = melted_df['Component'].map(component_names)

plt.figure(figsize=(12, 6))
sns.boxplot(data=melted_df, x='Component', y='Z-Score', palette='Set2', hue='Component')
plt.title('Distribution of Z-Scores by Component', fontsize=14)
plt.xlabel('Component', fontsize=12)
plt.ylabel('Z-Score', fontsize=12)

plt.axhline(0, color='gray', linestyle='--', alpha=0.7, label='Zero Reference')

plt.grid(axis='y', linestyle='--', alpha=0.3)

for i, comp in enumerate(component_names.values()):
    subset = melted_df[melted_df['Component'] == comp]['Z-Score']
    median = subset.median()
    plt.annotate(f'Median: {median:.2f}',
                 xy=(i, subset.min() - 0.2),
                 ha='center',
                 fontsize=9)

plt.tight_layout()
plt.show()

### 4.2 Distribution Summary

#### 4.2.1 Histogram of composite scores

Displays the overall distribution of the composite "well-resourced scores" across all analyzed SA2 regions. A histogram is generated to show the frequency of scores within different ranges, providing insight into whether scores are concentrated or spread out.

In [ ]:
scores_df = pd.read_sql(text("""
    SELECT
        public.sa2_summary.sa2_name,
        well_resourced_score,
        sa4_name_2021
    FROM public.sa2_summary
    JOIN public.sa2_regions USING (sa2_code)
"""), conn)

sns.set_style("whitegrid")
plt.figure(figsize=(10, 6))

plt.subplot(1, 2, 1)
sns.histplot(scores_df['well_resourced_score'], bins=15, kde=True, color='skyblue')
plt.title('Distribution of Well-Resourced Scores')
plt.xlabel('Score')
plt.ylabel('Number of SA2 Regions')

plt.tight_layout()
plt.show()

### 4.2.2 Overall stats (mean, median, std, IQR)
This code generates the overall statistics includes mean, median, spread which is standard deviation and interquartile range. It is done bymerging summary data with SA4 region names, compute the stats, then it groups the data by SA4 region and calculates the same metrics for each individual SA4. The results are merged into a single DataFrame and displayed as a table, allowing for a clear cross-region comparison.

In [106]:
summary_df = query(conn, "SELECT * FROM public.sa2_summary;")
sa2_regions_df = query(conn, "SELECT sa2_code, sa2_name, sa4_name_2021 FROM public.sa2_regions;")
summary_df = summary_df.merge(sa2_regions_df, on=["sa2_code", "sa2_name"], how="left")

q1 = summary_df["well_resourced_score"].quantile(0.25)
q3 = summary_df["well_resourced_score"].quantile(0.75)
iqr = q3 - q1

overall_stats = pd.DataFrame({
    "SA4 Region": ["Overall"],
    "Mean Score": [summary_df["well_resourced_score"].mean()],
    "Median Score": [summary_df["well_resourced_score"].median()],
    "Standard Deviation": [summary_df["well_resourced_score"].std()],
    "IQR": [iqr]
})

def calc_iqr(group):
    q1 = group.quantile(0.25)
    q3 = group.quantile(0.75)
    return q3 - q1

cross_zone_summary = summary_df.groupby("sa4_name_2021")["well_resourced_score"].agg(
    mean="mean",
    median="median",
    std="std",
    iqr=calc_iqr
).reset_index()

cross_zone_summary.columns = ["SA4 Region", "Mean Score", "Median Score", "Standard Deviation", "IQR"]

combined_summary = pd.concat([overall_stats, cross_zone_summary], ignore_index=True)

display(combined_summary)


Error encountered: 'NoneType' object has no attribute 'cursor'
Error encountered: 'NoneType' object has no attribute 'cursor'


AttributeError: 'NoneType' object has no attribute 'merge'

### 4.3 Cross-Zone Comparison

### 4.3 Cross-Zone Comparison

Compares the distribution of "well-resourced scores" across different SA4 zones. A boxplot visualization is used to illustrate differences or similarities in score distributions between these broader geographical areas, highlighting any systematic regional variations.

In [ ]:
scores_df = pd.read_sql(text("""
    SELECT
        public.sa2_summary.sa2_name,
        well_resourced_score,
        sa4_name_2021
    FROM public.sa2_summary
    JOIN public.sa2_regions USING (sa2_code)
"""), conn)

sns.set_style("whitegrid")
plt.figure(figsize=(10, 6))

plt.subplot(1, 2, 2)
sns.boxplot(data=scores_df, x='sa4_name_2021', y='well_resourced_score',
            hue='sa4_name_2021', palette='pastel')
plt.title('Score Distribution by SA4 Zone')
plt.xlabel('SA4 Zone')
plt.ylabel('Score')
plt.xticks(rotation=45, ha='right')

plt.tight_layout()
plt.show()

### 4.4 Noteworthy Patterns & Outliers

Identifies and visualizes the SA2 regions with the highest and lowest "well-resourced scores." A bar chart displays the top 5 and bottom 5 performing regions, allowing for a quick understanding of the extremes in resource distribution.

In [ ]:
top_5 = components_df.nlargest(5, 'well_resourced_score')
bottom_5 = components_df.nsmallest(5, 'well_resourced_score')

top_5['category'] = 'Top 5'
bottom_5['category'] = 'Bottom 5'
top_bottom_df = pd.concat([top_5, bottom_5])

top_bottom_df = top_bottom_df.sort_values('well_resourced_score')

plt.figure(figsize=(12, 6))
ax = sns.barplot(data=top_bottom_df,
                x='well_resourced_score',
                y='sa2_name',
                hue='category',
                palette={'Top 5': 'forestgreen', 'Bottom 5': 'firebrick'},
                dodge=False)

for i, v in enumerate(top_bottom_df['well_resourced_score']):
    ax.text(v + 0.01, i, f'{v:.2f}', va='center')

plt.title('Top and Bottom 5 Performing SA2 Regions', fontsize=14)
plt.xlabel('Well-Resourced Score', fontsize=12)
plt.ylabel('SA2 Region', fontsize=12)
plt.xlim(0, max(top_bottom_df['well_resourced_score']) * 1.1)
plt.grid(axis='x', linestyle='--', alpha=0.7)

plt.legend(title='', loc='lower right')

plt.tight_layout()
plt.show()

## 5. Correlation Analysis 

This section investigates the relationship between the "well-resourced scores" and socio-economic factors, specifically median income.

### 5.1 r & p-Value Calculation

Calculates the Pearson correlation coefficient (r) and p-value to assess the statistical relationship between "well-resourced scores" and median income. This is performed for all SA2 regions combined and then separately for SA2 regions within specific SA4 zones ("Sydney - Blacktown", "Sydney - Parramatta", "Sydney - Ryde") to identify if the correlation varies by larger geographical area.

In [ ]:
summary_df_raw = query(
    conn,
    """
    SELECT
      s.sa2_name,
      r.sa4_name_2021,
      s.well_resourced_score,
      i.median_income
    FROM sa2_summary AS s
    JOIN income       AS i USING (sa2_name)
    JOIN sa2_regions  AS r USING (sa2_name)
    """
)
print(f"Rows fetched from SQL: {len(summary_df_raw)}")

# Handle potential non-string median_income before .str.replace
summary_df = summary_df_raw.copy() # Work on a copy
summary_df["median_income"] = summary_df["median_income"].astype(str).str.replace(",", "")
summary_df["median_income"] = pd.to_numeric(
    summary_df["median_income"],
    errors="coerce"
)
print(f"Rows after median_income numeric conversion: {len(summary_df)}")
print(f"NaNs in median_income after coercion: {summary_df['median_income'].isna().sum()}")
print(f"NaNs in well_resourced_score: {summary_df['well_resourced_score'].isna().sum()}")

summary_df = summary_df.dropna(subset=["well_resourced_score", "median_income"])
print(f"Rows after dropping NaNs for correlation: {len(summary_df)}")

r_overall, p_overall = pearsonr(
    summary_df["well_resourced_score"],
    summary_df["median_income"]
)
print(f"Overall (n={len(summary_df)}): Pearson r = {r_overall:.3f}, p = {p_overall:.3f}")

regions = ["Sydney - Blacktown", "Sydney - Parramatta", "Sydney - Ryde"]
for region in regions:
    sub = summary_df[summary_df["sa4_name_2021"] == region]
    r, p = pearsonr(sub["well_resourced_score"], sub["median_income"])
    print(f"{region} (n={len(sub)}): Pearson r = {r:.3f}, p = {p:.3f}")

### 5.2 Regression plot of score vs. median income

Visualizes the relationship between "well-resourced scores" and median income using a scatter plot with an overlaid regression line. This plot helps to illustrate the direction and strength of the correlation, and the calculated Pearson correlation coefficient is annotated on the plot.

In [ ]:
corr_df = pd.read_sql(text("""
    SELECT 
        ss.sa2_name,
        ss.well_resourced_score,
        i.median_income,
        sr.sa4_name_2021
    FROM public.sa2_summary ss
    JOIN public.income i ON ss.sa2_code::text = i.sa2_code21::text
    JOIN public.sa2_regions sr ON ss.sa2_code = sr.sa2_code
"""), conn)

corr_df['well_resourced_score'] = pd.to_numeric(corr_df['well_resourced_score'], errors='coerce')
corr_df['median_income'] = pd.to_numeric(corr_df['median_income'], errors='coerce')

corr_df = corr_df.dropna(subset=['well_resourced_score', 'median_income'])

plt.figure(figsize=(10, 6))
sns.regplot(data=corr_df, 
            x='well_resourced_score', 
            y='median_income',
            scatter_kws={'alpha': 0.6},
            line_kws={'color': 'red'})

plt.title('Well-Resourced Score vs Median Income')
plt.xlabel('Well-Resourced Score')
plt.ylabel('Median Income ($)')
plt.grid(True)

# Calculate correlation coefficient
correlation = corr_df['well_resourced_score'].corr(corr_df['median_income'])
plt.annotate(f'Correlation: {correlation:.2f}', 
             xy=(0.05, 0.95), 
             xycoords='axes fraction',
             bbox=dict(boxstyle="round", facecolor='white', alpha=0.8))

plt.show()

## 6. Geographic Visualization 

### 6.1 Static Choropleth Map

Creates a static choropleth map displaying the "well-resourced scores" for SA2 regions within selected SA4 zones ("Sydney - Blacktown", "Sydney - Parramatta", "Sydney - Ryde"). Scores are represented by a color gradient, SA4 boundaries are overlaid for context, and a basemap is included to provide geographic reference.


In [ ]:
map_df = gpd.read_postgis(text("""
    SELECT
        sr.sa2_code,
        sr.sa2_name,
        sr.geometry,
        ss.well_resourced_score,
        sr.sa4_name_2021
    FROM public.sa2_regions sr
    JOIN public.sa2_summary ss ON sr.sa2_code = ss.sa2_code
    WHERE sr.sa4_name_2021 IN ('Sydney - Blacktown', 'Sydney - Parramatta', 'Sydney - Ryde')
"""), conn, geom_col='geometry')

fig, ax = plt.subplots(figsize=(12, 10))

map_df.plot(column='well_resourced_score',
            ax=ax,
            legend=True,
            legend_kwds={'label': "Well-Resourced Score", 'shrink': 0.5},
            cmap='RdYlGn',
            edgecolor='black',
            linewidth=0.3,
            missing_kwds={'color': 'lightgrey'})

sa4_boundaries = gpd.read_postgis(text("""
    SELECT
        sa4_name_2021,
        ST_Union(geometry) as geometry
    FROM public.sa2_regions
    WHERE sa4_name_2021 IN ('Sydney - Blacktown', 'Sydney - Parramatta', 'Sydney - Ryde')
    GROUP BY sa4_name_2021
"""), conn, geom_col='geometry')

sa4_boundaries.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=1.5)

for idx, row in sa4_boundaries.iterrows():
    ax.annotate(text=row['sa4_name_2021'],
                xy=row['geometry'].centroid.coords[0],
                ha='center',
                fontsize=10,
                bbox=dict(boxstyle="round", facecolor='white', alpha=0.7))

ctx.add_basemap(ax, crs=map_df.crs.to_string(), source=ctx.providers.CartoDB.Positron)

plt.title('Well-Resourced Scores by SA2 Region')
plt.axis('off')
plt.tight_layout()
plt.show()

### 6.2 Interactive Map

Generates an interactive choropleth map using Folium to visualize "well-resourced scores." The map includes features like hover tooltips displaying SA2 name, score, and SA4 region; multiple basemap options; a custom color legend; marker clusters for the top 5 regions; a title; layer control; fullscreen capability; and a measure tool for enhanced exploration.

In [ ]:
m = folium.Map(location=[-33.8, 151.0], zoom_start=11, tiles='cartodbpositron')

folium.TileLayer('cartodbpositron', name='Light Map').add_to(m)
folium.TileLayer('cartodbdark_matter', name='Dark Map').add_to(m)
folium.TileLayer('OpenStreetMap', name='OpenStreetMap').add_to(m)
folium.TileLayer('Stamen Terrain', name='Terrain').add_to(m)

def get_color(score):
    if score is None:
        return '#CCCCCC'
    elif score >= 0.8:
        return '#1a9850'
    elif score >= 0.6:
        return '#91cf60'
    elif score >= 0.4:
        return '#ffffbf'
    elif score >= 0.2:
        return '#fc8d59'
    else:
        return '#d73027'

def style_function(feature):
    score = feature['properties']['well_resourced_score']
    return {
        'fillColor': get_color(score),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7
    }

def highlight_function(feature):
    return {
        'weight': 3,
        'color': '#666',
        'dashArray': '',
        'fillOpacity': 0.9
    }

choropleth = folium.GeoJson(
    map_df,
    name='Well-Resourced Scores',
    style_function=style_function,
    highlight_function=highlight_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['sa2_name', 'well_resourced_score', 'sa4_name_2021'],
        aliases=['Area:', 'Score:', 'Region:'],
        localize=True,
        sticky=True,
        labels=True,
        style="""
            background-color: #F0EFEF;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px 3px 3px rgba(0,0,0,0.4);
            font-size: 14px;
            padding: 10px;
        """
    )
).add_to(m)

legend_html = '''
    <div style="position: fixed;
        bottom: 50px; right: 50px; width: 180px; height: 170px;
        border:2px solid grey; z-index:9999; font-size:14px;
        background-color: white; padding: 10px; border-radius: 5px">
    <p style="margin-top: 0; margin-bottom: 5px;"><b>Well-Resourced Score</b></p>
    <div style="display: flex; align-items: center; margin-bottom: 5px;">
        <span style="background-color:#1a9850; width:20px; height:20px; display:inline-block; margin-right:5px;"></span>
        <span>0.8 - 1.0</span>
    </div>
    <div style="display: flex; align-items: center; margin-bottom: 5px;">
        <span style="background-color:#91cf60; width:20px; height:20px; display:inline-block; margin-right:5px;"></span>
        <span>0.6 - 0.8</span>
    </div>
    <div style="display: flex; align-items: center; margin-bottom: 5px;">
        <span style="background-color:#ffffbf; width:20px; height:20px; display:inline-block; margin-right:5px;"></span>
        <span>0.4 - 0.6</span>
    </div>
    <div style="display: flex; align-items: center; margin-bottom: 5px;">
        <span style="background-color:#fc8d59; width:20px; height:20px; display:inline-block; margin-right:5px;"></span>
        <span>0.2 - 0.4</span>
    </div>
    <div style="display: flex; align-items: center; margin-bottom: 5px;">
        <span style="background-color:#d73027; width:20px; height:20px; display:inline-block; margin-right:5px;"></span>
        <span>0.0 - 0.2</span>
    </div>
    </div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

marker_cluster = MarkerCluster(name="Top Regions").add_to(m)

top_regions = map_df.nlargest(5, 'well_resourced_score')
for idx, row in top_regions.iterrows():
    centroid = row['geometry'].centroid
    popup_text = f"""
    <b>{row['sa2_name']}</b><br>
    Score: {row['well_resourced_score']:.2f}<br>
    Region: {row['sa4_name_2021']}
    """

    folium.Marker(
        location=[centroid.y, centroid.x],
        popup=folium.Popup(popup_text, max_width=300),
        icon=folium.Icon(color='green', icon='star', prefix='fa'),
        tooltip=f"Top Region: {row['sa2_name']}"
    ).add_to(marker_cluster)

title_html = '''
    <div style="position: fixed;
        top: 10px; left: 50%; transform: translateX(-50%);
        z-index:9999; font-size:18px; font-weight: bold;
        background-color: white; padding: 10px; border-radius: 5px;
        border:2px solid grey; text-align: center;">
        Well-Resourced Scores by SA2 Region - Sydney
    </div>
'''
m.get_root().html.add_child(folium.Element(title_html))

folium.LayerControl(collapsed=False).add_to(m)

from folium.plugins import Fullscreen
Fullscreen().add_to(m)

from folium.plugins import MeasureControl
m.add_child(MeasureControl(position='topleft', primary_length_unit='kilometers'))

m

## 7. Scrutiny of Results

### 7.1 Sensitivity testing 
Assesses the impact of individual components (z_business, z_stops, z_schools, z_poi) on the final "well-resourced score" and its ranking. This involves:
1. Calculating the direct Pearson correlation of each component's z-score with the composite score.
2. Performing a leave-one-out analysis: each component is temporarily removed, the composite score is recalculated, and the Spearman rank correlation between the original ranking and the new ranking is computed.
3. Comparing distributional statistics (mean, median, standard deviation) of the original composite score with those of scores recalculated after dropping each component.


In [ ]:
sql = """
SELECT
  s.sa2_name,
  r.sa4_name_2021,
  s.well_resourced_score,
  s.z_business,
  s.z_stops,
  s.z_schools,
  s.z_poi,
  i.median_income
FROM sa2_summary AS s
JOIN income     AS i USING (sa2_name)
JOIN sa2_regions AS r USING (sa2_name)
"""
df = query(conn, sql).dropna()

df["median_income"] = df["median_income"].str.replace(",", "")
df["median_income"] = pd.to_numeric(df["median_income"], errors="coerce")
df = df.dropna(subset=["well_resourced_score", "median_income"])

components    = ["z_business", "z_stops", "z_schools", "z_poi"]
composite_col = "well_resourced_score"

print("1) Direct Pearson correlations with final score:")
for comp in components:
    r, p = pearsonr(df[comp], df[composite_col])
    print(f"   {comp:>12s} → r = {r:.3f}, p = {p:.3f}")

orig_rank = df[composite_col].rank(method="average")
print("\n2) Leave-one-out Spearman rank correlations:")
for comp in components:
    others   = [c for c in components if c != comp]
    raw_sum  = df[others].sum(axis=1)
    mod_score = 1 / (1 + np.exp(-raw_sum))
    mod_rank  = mod_score.rank(method="average")
    rho, p    = spearmanr(orig_rank, mod_rank)
    print(f"   without {comp:>8s} → ρ = {rho:.3f}, p = {p:.3f}")

orig_stats = (
    df[composite_col].mean(),
    df[composite_col].median(),
    df[composite_col].std()
)
print("\n3) Distribution statistics comparison (mean, median, std):")
print(f"   original       : mean = {orig_stats[0]:.3f}, median = {orig_stats[1]:.3f}, std = {orig_stats[2]:.3f}")
for comp in components:
    others    = [c for c in components if c != comp]
    raw_sum   = df[others].sum(axis=1)
    mod_score = 1 / (1 + np.exp(-raw_sum))
    stats     = (mod_score.mean(), np.median(mod_score), mod_score.std())
    print(f"   without {comp:>8s} : mean = {stats[0]:.3f}, median = {stats[1]:.3f}, std = {stats[2]:.3f}")

## OUR UNSTRUCTURED CODES ⤵️
-- MOVE UP YOUR CODE INTO THEIR SECTIONS --

## Setup and Connection
### Imports  
Load all Python libraries used in Task 1:  
* **pandas / geopandas** for data wrangling  
* **sqlalchemy + psycopg2** to talk to Postgres/PostGIS  
* **geoalchemy2.Geometry** helper so geopandas can write geometries  
* a few plotting + shapely utilities for quick inspection


In [58]:
import pandas as pd 
import geopandas as gpd 
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt 

In [59]:
import json
import pandas as pd
from sqlalchemy import create_engine, text
credentials = "Credentials.json"
def pgconnect(credential_filepath, db_schema="public"):
    """
    Reads credentials.json, prefers the 'database' key, then 'dbname',
    then falls back to the user. Returns (engine, connection).
    """
    with open(credential_filepath, "r") as f:
        creds = json.load(f)

    host   = creds["host"]
    port   = creds["port"]
    user   = creds["user"]
    pw     = creds["password"]
    dbname = creds.get("database") or creds.get("dbname") or user

    url = f"postgresql+psycopg2://{user}:{pw}@{host}:{port}/{dbname}"

    try:
        engine = create_engine(url, echo=False)
        conn   = engine.connect()
        print(f"Connected successfully to '{dbname}'.")
    except Exception as e:
        print("Unable to connect to the database:")
        print(e)
        engine, conn = None, None

    return engine, conn

def query(engine, sqlcmd, args=None, df=True):
    """
    Convenience wrapper around pandas.read_sql_query or raw execute.
    - If df=True: returns a DataFrame via pandas.read_sql_query()
    - Else: runs a raw execute() on engine and returns fetched results.
    """
    try:
        if df:
            return pd.read_sql_query(sqlcmd, engine, params=args)
        else:
            result = engine.execute(text(sqlcmd), args).fetchall()
            return result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered:", e)
        return None


### Connect to the database  
Runs `pgconnect()` – if you see **“Connected successfully.”** the engine + connection are ready.


In [61]:
db, conn = pgconnect(credentials)

Unable to connect to the database:
(psycopg2.OperationalError) connection to server at "localhost" (::1), port 5432 failed: FATAL:  database "localhost" does not exist

(Background on this error at: https://sqlalche.me/e/20/e3q8)


### Sanity check: confirm PostGIS is installed  
Queries `PostGIS_Version()`; if it returns a version string, spatial functions are available.


In [63]:
query(conn, "select PostGIS_Version()")

Error encountered: 'NoneType' object has no attribute 'cursor'


## Load Datas
###  Read Business, Population, Income CSVs 
Read the three non-spatial CSV files into pandas dataframes.  
At this point we only preview them; they get written to Postgres later with `to_sql()`.


In [65]:
business = pd.read_csv("data/Businesses.csv")
business.head()

,industry_code,industry_name,sa2_code,sa2_name,0_to_50k_businesses,50k_to_200k_businesses,200k_to_2m_businesses,2m_to_5m_businesses,5m_to_10m_businesses,10m_or_more_businesses,total_businesses
0,A,"Agriculture, Forestry and Fishing",101021007,Braidwood,136,92,63,4,0,0,296
1,A,"Agriculture, Forestry and Fishing",101021008,Karabar,6,3,0,0,0,0,9
2,A,"Agriculture, Forestry and Fishing",101021009,Queanbeyan,6,4,3,0,0,3,15
3,A,"Agriculture, Forestry and Fishing",101021010,Queanbeyan - East,0,3,0,0,0,0,3
4,A,"Agriculture, Forestry and Fishing",101021012,Queanbeyan West - Jerrabomberra,7,4,5,0,0,0,16


In [66]:
income = pd.read_csv("data/income.csv")
income.head()

,sa2_code21,sa2_name,earners,median_age,median_income,mean_income
0,101021007,Braidwood,2467,51,46640,68904
1,101021008,Karabar,5103,42,65564,69672
2,101021009,Queanbeyan,7028,39,63528,69174
3,101021010,Queanbeyan - East,3398,39,66148,74162
4,101021012,Queanbeyan West - Jerrabomberra,8422,44,78630,91981


In [67]:
population = pd.read_csv("data/Population.csv")
population.head()

,sa2_code,sa2_name,0-4_people,5-9_people,10-14_people,15-19_people,20-24_people,25-29_people,30-34_people,35-39_people,...,45-49_people,50-54_people,55-59_people,60-64_people,65-69_people,70-74_people,75-79_people,80-84_people,85-and-over_people,total_people
0,102011028,Avoca Beach - Copacabana,424,522,623,552,386,222,306,416,...,572,602,570,520,464,369,226,142,70,7530
1,102011029,Box Head - MacMasters Beach,511,666,702,592,461,347,420,535,...,749,749,794,895,863,925,603,331,264,11052
2,102011030,Calga - Kulnura,200,225,258,278,274,227,214,286,...,325,436,422,397,327,264,190,100,75,4748
3,102011031,Erina - Green Point,683,804,880,838,661,502,587,757,...,859,882,901,930,917,1065,976,773,1028,14803
4,102011032,Gosford - Springfield,1164,1044,1084,1072,1499,1864,1750,1520,...,1330,1241,1377,1285,1166,949,664,476,537,21346


### Define Table Schemas in SQL  
* Raw SQL (**cell 7**) creates three empty spatial tables  
  (`sa2_regions`, `stops`, `schools`) with geometry columns, SRID 4326,  



In [69]:
from sqlalchemy import text 
create_sa2_sql = """
DROP TABLE IF EXISTS public.sa2_regions;
CREATE TABLE public.sa2_regions (
    sa2_code_2021 TEXT PRIMARY KEY,
    sa2_name_2021 TEXT,
    geometry      GEOMETRY(MULTIPOLYGON,4326)
);
CREATE INDEX IF NOT EXISTS sa2_regions_geom_idx
    ON public.sa2_regions USING GIST (geometry);

DROP TABLE IF EXISTS public.stops;
CREATE TABLE public.stops (
    stop_id   TEXT PRIMARY KEY,
    stop_name TEXT,
    geometry  GEOMETRY(POINT,4326)
);
CREATE INDEX IF NOT EXISTS stops_geom_idx
    ON public.stops USING GIST (geometry);

"""

conn.execute(text(create_sa2_sql))


AttributeError: 'NoneType' object has no attribute 'execute'

### Load CSVs into Postgres with Pandas
  primary keys, and GIST indexes.  
* **cell 8** loops through *Businesses / Population / Income* dataframes  
  and writes them to Postgres with `to_sql()`, then adds b-tree indexes on `sa2_code`.

In [ ]:
for csv_file, table in [
    ("data/Businesses.csv", "businesses"),
    ("data/Population.csv", "population"),
    ("data/Income.csv", "income"),
]:
    df = pd.read_csv(csv_file)
    df.drop_duplicates(inplace=True)
    df.to_sql(
        table,
        conn,
        schema = "public",
        if_exists = "replace",
        index = False,
        method = "multi")
    print(f"Loaded {table}: {len(df)} rows")


In [ ]:
# Method A: using your helper
cols = query(conn, """
    SELECT column_name
    FROM information_schema.columns
    WHERE table_name='income'
      AND table_schema='public';
""")
print(cols)

# Method B: manually fetch
result = conn.execute(text("""
    SELECT column_name
    FROM information_schema.columns
    WHERE table_name='income'
      AND table_schema='public';
"""))
print([row[0] for row in result.fetchall()])

### Add b-tree indexes on `sa2_code`  
Speeds up joins between attribute tables and spatial layers.


In [ ]:
from sqlalchemy import text

conn.execute(text("""
    CREATE INDEX IF NOT EXISTS businesses_sa2_idx
      ON public.businesses (sa2_code);
    CREATE INDEX IF NOT EXISTS population_sa2_idx
      ON public.population (sa2_code);
    CREATE INDEX IF NOT EXISTS income_sa2_idx
      ON public.income (sa2_code21);
"""))
print("Indexes created on sa2_code for all three tables.")



## 3. Load Spatial Tables
### Load SA2 boundaries shapefile  
* Reads `SA2_2021_AUST_GDA2020.shp` with `gpd.read_file()`.  
* Keeps only code, name, geometry; reprojects to WGS-84 (EPSG 4326).


In [ ]:
# --- Load SA2 boundaries WITH SA4 + GCC columns --------------------------
import geopandas as gpd
from geoalchemy2 import Geometry
from sqlalchemy import text

# 1. Read shapefile and keep the extra admin columns
sa2 = (
    gpd.read_file("data/SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp")[
        ["SA2_CODE21", "SA2_NAME21", "SA4_NAME21", "GCC_NAME21", "geometry"]
    ]
    .rename(
        columns={
            "SA2_CODE21": "sa2_code",
            "SA2_NAME21": "sa2_name",
            "SA4_NAME21": "sa4_name_2021",
            "GCC_NAME21": "gcc_name_2021",
        }
    )
    .to_crs(4326)
    .drop_duplicates()
)

# 2. Re-create the table from scratch
conn.execute(text("DROP TABLE IF EXISTS public.sa2_regions;"))

sa2.to_postgis(
    "sa2_regions", conn,
    schema="public",
    if_exists="replace",            # overwrite
    index=False,
    dtype={"geometry": Geometry("MULTIPOLYGON", 4326)}
)

# 3. Add PK and spatial index
conn.execute(text("""
ALTER TABLE public.sa2_regions
  ADD CONSTRAINT sa2_regions_pkey PRIMARY KEY (sa2_code);
CREATE INDEX sa2_regions_geom_idx
  ON public.sa2_regions USING GIST (geometry);
"""))

print(f"Reloaded {len(sa2):,} SA2 polygons with SA4 & GCC info.")


### Create Spatial Table for SA2 
Drops any old copy and recreates `public.sa2_regions` with  
`GEOMETRY(MULTIPOLYGON,4326)`, primary key `sa2_code_2021`, and GIST index.


In [ ]:
ddl = """
DROP TABLE IF EXISTS public.sa2_regions;
CREATE TABLE public.sa2_regions (
    sa2_code TEXT PRIMARY KEY,
    sa2_name TEXT,
    geometry GEOMETRY(MULTIPOLYGON,4326)
);
CREATE INDEX IF NOT EXISTS sa2_regions_geom_idx
  ON public.sa2_regions USING GIST (geometry);
"""
conn.execute(text(ddl))
print("Spatial table sa2_regions created.")

### Insert SA2 to PostGIS 
`to_postgis()` streams the GeoDataFrame into the empty table.


In [ ]:
sa2.to_postgis(
    "sa2_regions", conn,
    schema="public",
    if_exists="replace",   
    index=False,
    dtype={"geometry": Geometry("MULTIPOLYGON", 4326)}
)
print(f"Inserted {len(sa2):,} SA2 rows.")

### Ensure SA2 Table Index Exists
PL/pgSQL block only adds them if missing (useful if the cell is rerun).


In [ ]:
from sqlalchemy import text

conn.execute(text("""
DO $$
BEGIN
  -- add PK only if it doesn't exist
  IF NOT EXISTS (
      SELECT 1 FROM pg_constraint
      WHERE conname = 'sa2_regions_pkey'
  ) THEN
      ALTER TABLE public.sa2_regions
        ADD CONSTRAINT sa2_regions_pkey PRIMARY KEY (sa2_code);
  END IF;

  -- add GIST index only if it doesn't exist
  IF NOT EXISTS (
      SELECT 1 FROM pg_indexes
      WHERE schemaname = 'public'
        AND tablename  = 'sa2_regions'
        AND indexname  = 'sa2_regions_geom_idx'
  ) THEN
      CREATE INDEX sa2_regions_geom_idx
        ON public.sa2_regions USING GIST (geometry);
  END IF;
END $$;
"""))

print("Primary key and GIST index verified/created.")



### Select SA2 polygons for my SA4 zone  
The ABS shapefile stores the SA4 each SA2 belongs to.  
Here we keep only SA2s that are:
* inside the **Greater Sydney** GCC, and  
* inside the SA4 zone we choose (set in `TARGET_SA4`).  

The resulting GeoDataFrame `my_sa2` will drive every later step
(spatial joins, POI API calls, scoring …).


In [ ]:
pd.read_sql("""
SELECT DISTINCT sa4_name_2021
FROM   public.sa2_regions
WHERE  gcc_name_2021 = 'Greater Sydney'
ORDER  BY sa4_name_2021;
""", conn)


## Team Memember Hardik Chojar Chose "Sydney - Parramatta"

In [ ]:
TARGET_SA4 = "Sydney - Parramatta"     

my_sa2 = gpd.read_postgis(
    """
    SELECT sa2_code,
           sa2_name,
           geometry
    FROM   public.sa2_regions
    WHERE  gcc_name_2021 = 'Greater Sydney'
      AND  sa4_name_2021 = %(sa4)s
    """,
    conn,
    geom_col="geometry",
    params={"sa4": TARGET_SA4}
)

print(f"{len(my_sa2)} SA2s in {TARGET_SA4}")


## 4. Load Public Transport Stops
### Load GTFS Stops and Convert to Geometry  
* Reads `Stops.txt`, constructs a GeoDataFrame from lon/lat via  
  `gpd.points_from_xy()`.  
* Appends rows to `public.stops`.


In [ ]:
import pandas as pd, geopandas as gpd
from geoalchemy2 import Geometry

# build GeoDataFrame
stops = pd.read_csv("data/Stops.txt")
stops.drop_duplicates(inplace=True)

stops_gdf = gpd.GeoDataFrame(
    stops[["stop_id","stop_name"]],
    geometry=gpd.points_from_xy(stops.stop_lon, stops.stop_lat),
    crs=4326
)

# append rows
stops_gdf.to_postgis("stops", conn,
                     schema="public",
                     if_exists="append",   # structure already created
                     index=False,
                     dtype={"geometry": Geometry("POINT",4326)})

print(f"Inserted {len(stops_gdf):,} stops")


## 5. Load School Catchments
### Merge Primary, Secondary, and Future Catchments
* Reads each shapefile, renames `USE_ID → school_id` and `CATCH_TYPE → stage`,  
  tags `sector='Government'`, concatenates.  
* Recreates `public.schools` with a surrogate key (`gid SERIAL`) + GIST index,  
  then inserts all polygons.


In [ ]:
import geopandas as gpd
from geoalchemy2 import Geometry
import pandas as pd
from sqlalchemy import text

def read_catch(path):
    gdf = gpd.read_file(path).to_crs(4326)
    gdf = gdf.rename(columns={
        "USE_ID": "school_id",
        "CATCH_TYPE": "stage"
    })
    gdf["sector"] = "Government"
    return gdf[["school_id", "sector", "stage", "geometry"]]

# read each shapefile
gdf_primary   = read_catch("data/catchments/catchments_primary.shp")
gdf_secondary = read_catch("data/catchments/catchments_secondary.shp")
gdf_future    = read_catch("data/catchments/catchments_future.shp")

schools = pd.concat([gdf_primary, gdf_secondary, gdf_future], ignore_index=True)

schools.drop_duplicates(inplace=True)

print(f"Total school polygons: {len(schools):,}")

# recreate spatial table with surrogate PK
conn.execute(text("""
DROP TABLE IF EXISTS public.schools;
CREATE TABLE public.schools (
    gid       SERIAL PRIMARY KEY,          -- surrogate key
    school_id TEXT,
    sector    TEXT,
    stage     TEXT,
    geometry  GEOMETRY(MULTIPOLYGON,4326)
);
CREATE INDEX schools_geom_idx
  ON public.schools USING GIST (geometry);
"""))

# insert rows
schools.to_postgis("schools", conn, schema="public",
                   if_exists="append", index=False,
                   dtype={"geometry": Geometry("MULTIPOLYGON",4326)})

print("School catchments loaded with surrogate primary key.")


## 6. Final Validation
### Row Counts & SRID Check  
* Row counts for all six tables (ensures data really loaded).  
* SRID confirmation – every geometry table is EPSG 4326.  
Both checks pass ⇒ Task 1 satisfied.


In [ ]:
import pandas as pd

pd.read_sql("""
SELECT 'sa2_regions' AS table_name, COUNT(*) AS rows FROM public.sa2_regions
UNION ALL
SELECT 'stops',        COUNT(*) FROM public.stops
UNION ALL
SELECT 'schools',      COUNT(*) FROM public.schools
UNION ALL
SELECT 'businesses',   COUNT(*) FROM public.businesses
UNION ALL
SELECT 'population',   COUNT(*) FROM public.population
UNION ALL
SELECT 'income',       COUNT(*) FROM public.income;
""", conn)




In [ ]:
pd.read_sql("""
SELECT f_table_name AS table_name, srid
FROM   geometry_columns
WHERE  f_table_schema = 'public';
""", conn)


In [ ]:
import pandas as pd
from sqlalchemy import text

# engine was returned by your pgconnect() call
# replace `db` with whatever variable holds your SQLAlchemy engine
print("engine    →", db.engine.url)    
print("conn      →", conn.connection.get_dsn_parameters())

# and run
print(pd.read_sql(text("SELECT current_database()"), db).iloc[0,0])


# Task 2
## Part 1
Here are the function that returns all points of interests from the API within a specified bounding box of coordinates.

In [ ]:
import requests
import pandas as pd

def get_pois_from_api(minx, miny, maxx, maxy):
    
    url = "https://maps.six.nsw.gov.au/arcgis/rest/services/public/NSW_POI/MapServer/0/query"
    
    params = {
        "where": "1=1",
        "geometry": f"{minx},{miny},{maxx},{maxy}",
        "geometryType": "esriGeometryEnvelope",
        "spatialRel": "esriSpatialRelIntersects",
        "outFields": "*",
        "returnGeometry": "true",
        "f": "json"
    }
    
    response = requests.get(url, params=params)

    if response.status_code != 200:
        print("Error fetching data:", response.status_code)
        return pd.DataFrame(), {}
    
    result = response.json()

    if "features" not in result:
        return pd.DataFrame(), result
    
    pois = []
    
    for feature in result["features"]:
        attr = feature["attributes"]
        geom = feature.get("geometry", {})
        
        pois.append({
            "POIName": attr.get("poiname", None),
            "POIType": attr.get("poitype", None),
            "POIGroup": attr.get("poigroup", None),
            "POILabel": attr.get("poilabel", None),
            "POIAltLabel": attr.get("poialtlabel", None),
            "POILabelType": attr.get("poilabeltype", None),
            "Longitude": geom.get("x", None),
            "Latitude": geom.get("y", None)
        })
    
    return pd.DataFrame(pois), result


In [ ]:
import requests
import pandas as pd

def get_pois_from_api(minx, miny, maxx, maxy):
    
    url = "https://maps.six.nsw.gov.au/arcgis/rest/services/public/NSW_POI/MapServer/0/query"
    
    params = {
        "where": "1=1",
        "geometry": f"{minx},{miny},{maxx},{maxy}",
        "geometryType": "esriGeometryEnvelope",
        "spatialRel": "esriSpatialRelIntersects",
        "outFields": "*",
        "returnGeometry": "true",
        "f": "json"
    }
    
    response = requests.get(url, params=params)

    if response.status_code != 200:
        print("Error fetching data:", response.status_code)
        return pd.DataFrame(), {}
    
    result = response.json()

    if "features" not in result:
        return pd.DataFrame(), result
    
    pois = []
    
    for feature in result["features"]:
        attr = feature["attributes"]
        geom = feature.get("geometry", {})
        
        pois.append({
            "POIName": attr.get("poiname", None),
            "POIType": attr.get("poitype", None),
            "POIGroup": attr.get("poigroup", None),
            "POILabel": attr.get("poilabel", None),
            "POIAltLabel": attr.get("poialtlabel", None),
            "POILabelType": attr.get("poilabeltype", None),
            "Longitude": geom.get("x", None),
            "Latitude": geom.get("y", None)
        })
    
    return pd.DataFrame(pois), result


## Part 2
loop that cycles through each SA2 region within your selected SA4 region, waits a second before executing, then runs
the function for that region’s bounding box, to find all points of interest within that SA2 region.

In [ ]:
import time
from sqlalchemy import text, String, Integer, Float, DateTime

selected_sa4 = ["Sydney - Blacktown", "Sydney - Parramatta", "Sydney - Ryde"]
selected_sa4_str = ", ".join(f"'{sa4}'" for sa4 in selected_sa4)

sa2_df = pd.read_sql(f"""
SELECT sa2_code, sa2_name, sa4_name_2021, ST_Extent(geometry) AS bbox
FROM public.sa2_regions
WHERE sa4_name_2021 IN ({selected_sa4_str})
GROUP BY sa2_code, sa2_name, sa4_name_2021
""", conn)

def bbox_parse(bbox_str):
    bbox_str = bbox_str.replace("BOX(", "").replace(")", "")
    min_point, max_point = bbox_str.split(",")
    minx, miny = map(float, min_point.strip().split(" "))
    maxx, maxy = map(float, max_point.strip().split(" "))
    return minx, miny, maxx, maxy

sa2_df[["min_lon", "min_lat", "max_lon", "max_lat"]] = sa2_df["bbox"].apply(lambda x: pd.Series(bbox_parse(x)))

results = []

for index, row in sa2_df.iterrows():
    sa2_code = row["sa2_code"]
    sa2_name = row["sa2_name"]
    min_lon = row["min_lon"]
    min_lat = row["min_lat"]
    max_lon = row["max_lon"]
    max_lat = row["max_lat"]

    print(f"Fetching POIs for SA2: {sa2_name}")

    poi_df, result_p = get_pois_from_api(min_lon, min_lat, max_lon, max_lat)
    # print(result_p)

    if poi_df.empty:
        continue

    poi_df["SA2_CODE"] = sa2_code
    poi_df["SA2_NAME"] = sa2_name
    results.append(poi_df)

    time.sleep(1)

print("Finished fetching POIs.")

if results:
    final_poi_df = pd.concat(results, ignore_index=True)
    print(f"Total POIs fetched: {len(final_poi_df)}")
    display(final_poi_df.head())
    

    final_poi_df.to_sql("nsw_poi", db, if_exists="replace", index=False, dtype={
        "POIName": String(),
        "POIType": String(),
        "POIGroup": Integer(),
        "POILabel": String(),
        "POIAltLabel": String(),
        "POILabelType": String(),
        "Longitude": Float(),
        "Latitude": Float(),
        "SA2_CODE": String(),
        "SA2_NAME": String(),
        "timestamp": String()   # → or DateTime() if you want timestamp
    })

    print("POIs successfully inserted into table 'nsw_poi'.")
else:
    print("No POIs found.")

# Task 3

## Businesses 
We focus on industries that directly provide everyday goods, services, and amenities to local residents.  To that end, we exclude:
	
    •	Heavy-industry & utilities (Mining, Manufacturing, Agriculture, Electricity/Gas/Water/Waste, Construction)
These sectors typically locate outside dense population centers and don’t reflect the local availability of shops, clinics, or leisure services.
	
    •	Public Administration & Safety
Although essential, these are government-run functions rather than private-sector businesses, and so are better handled by dedicated public-sector layers.
	
    •	Administrative & Support Services
Roles like cleaning, security or back-office operations don’t signal consumer-facing amenities; they rarely indicate local resource richness from a resident’s perspective.

In [ ]:
from sqlalchemy import text

sql = text("""
DROP TABLE IF EXISTS public.sa2_summary;

CREATE TABLE public.sa2_summary AS
SELECT
  sr.sa2_code,
  sr.sa2_name,
  SUM(b.total_businesses)
    / (pop.total_people   / 1000.0) AS business_count
FROM businesses b
  JOIN sa2_regions    sr  USING (sa2_name)
  JOIN population pop USING (sa2_name)
WHERE sr.sa4_name_2021 IN (
    'Sydney - Blacktown',
    'Sydney - Parramatta',
    'Sydney - Ryde'
  )
  AND b.industry_name IN (
    'Health Care and Social Assistance',
    'Education and Training',
    'Retail Trade',
    'Accommodation and Food Services',
    'Transport, Postal and Warehousing',
    'Financial and Insurance Services',
    'Professional, Scientific and Technical Services',
    'Arts and Recreation Services',
    'Rental, Hiring and Real Estate Services'
  )
  AND total_people >= 100
GROUP BY
  sr.sa2_code,
  sr.sa2_name,
  pop.total_people
""")

conn.execute(sql)


summary_df = query(conn, "SELECT * FROM public.sa2_summary;")
print(summary_df.head())

## Stops
We capture public‐transport accessibility by counting how many transit stops lie within each SA2.  A higher stops_count indicates better local connectivity and ease of movement for residents.  

In [ ]:
sql = text("""
-- 1) Add the new column if it doesn't already exist
ALTER TABLE public.sa2_summary
ADD COLUMN IF NOT EXISTS stops_count INTEGER;

-- 2) Update that column by counting stops per SA2
UPDATE public.sa2_summary AS summ
SET stops_count = sub.stops_count
FROM (
  SELECT
    sr.sa2_name,
    COUNT(*) AS stops_count
  FROM sa2_regions AS sr
  JOIN stops AS s
    ON ST_Contains(sr.geometry, s.geometry)
  WHERE
    sr.sa4_name_2021 IN (
      'Sydney - Blacktown',
      'Sydney - Parramatta',
      'Sydney - Ryde'
    )
  GROUP BY
    sr.sa2_name
) AS sub
WHERE summ.sa2_name = sub.sa2_name;
""")

conn.execute(sql)

summary_df = query(
    conn,
    "SELECT * FROM public.sa2_summary;"
)
print(summary_df.head())

## Schools
To gauge local education access, we count how many distinct school‐catchment polygons actually overlap each SA2, then express that as “catchments per 1 000 young people.”  We:
	
    •	Exclude any SA2 with fewer than 100 residents aged 0–19, to avoid unstable rates.
	•	Divide the raw catchment count by (young_pop / 1 000) so that areas with very few kids don’t produce huge per-capita spikes

In [ ]:
sql = text("""
-- 1) Add the new column if it doesn't already exist
ALTER TABLE public.sa2_summary
ADD COLUMN IF NOT EXISTS catchments_count DOUBLE PRECISION;

-- 2) Update that column by computing catchments per 1 000 young people
WITH pop_young AS (
  SELECT
    sa2_name,
    (
      "0-4_people"
    + "5-9_people"
    + "10-14_people"
    + "15-19_people"
    ) AS young_pop
  FROM population
)
UPDATE public.sa2_summary AS summ
SET catchments_count = sub.catchments_count
FROM (
  SELECT
    sr.sa2_name,
    COUNT(*)::float
      / (pyoung.young_pop / 1000.0) AS catchments_count
  FROM schools      AS s
  JOIN sa2_regions AS sr
    ON  ST_Intersects(sr.geometry, s.geometry)
    AND NOT ST_Touches(   sr.geometry, s.geometry)
  JOIN pop_young   AS pyoung
    ON pyoung.sa2_name = sr.sa2_name
  WHERE
    sr.sa4_name_2021 IN (
      'Sydney - Blacktown',
      'Sydney - Parramatta',
      'Sydney - Ryde'
    )
    AND pyoung.young_pop >= 100
  GROUP BY
    sr.sa2_name,
    pyoung.young_pop
) AS sub
WHERE summ.sa2_name = sub.sa2_name;
""")

conn.execute(sql)

summary_df = query(
    conn,
    "SELECT * FROM public.sa2_summary;"
)
print(summary_df.head())

## NSW POI
To measure local amenity density, we count only those points of interest that represent community-facing services and everyday resources.  We exclude:
	
    •	Industrial or utility facilities (Filtration Plant, Sewage Works, Pumping Station) as they don’t serve residents directly.
	•	Large‐scale commercial or tourism sites (Tourist Park, Silo – Commercial, Quarry – Open Cut) which typically lie outside residential cores.
	•	Very broad or redundant places (City, Suburb, Locality) that don’t convey true amenity density.
	•	Niche or infrequent-use sites (Motor Racing Track, Helipad, Gaol) which aren’t regular community resources.

In [ ]:
sql = text("""
-- 1) Add the new column if it doesn't already exist
ALTER TABLE public.sa2_summary
ADD COLUMN IF NOT EXISTS poi_count INTEGER;

-- 2) Update that column by counting POIs per SA2
UPDATE public.sa2_summary AS summ
SET poi_count = sub.poi_count
FROM (
  SELECT
    sr.sa2_name,
    COUNT(*) AS poi_count
  FROM nsw_poi AS n
  JOIN sa2_regions AS sr
    ON sr.sa2_name = n."SA2_NAME"
  WHERE
    sr.sa4_name_2021 IN (
      'Sydney - Blacktown',
      'Sydney - Parramatta',
      'Sydney - Ryde'
    )
    AND n."POIType" IN (
      'Library',
      'Park',
      'Community Facility',
      'Child Care Centre',
      'Primary School',
      'High School',
      'General Hospital',
      'Community Medical Centre',
      'Transport Interchange',
      'Post Office',
      'Sports Centre / Sports Field',
      'Art Gallery / Museum'
    )
  GROUP BY
    sr.sa2_name
) AS sub
WHERE summ.sa2_name = sub.sa2_name;
""")

conn.execute(sql)

summary_df = query(
    conn,
    "SELECT * FROM public.sa2_summary;"
)
print(summary_df.head())

## Z scores and Final Sigmoid Score
To bring all four metrics (business, stops, schools, POIs) onto a common scale and combine them into one index, we:
	
    • Standardize each metric by converting raw counts into z-scores, where mu and sigma are the mean and population standard deviation of that metric across your selected SA2s.
	
    • Sum the four z-scores into a single value, so positive values indicate above-average resource levels and negatives indicate below-average.
	
    • Apply a sigmoid transform, to squash the combined score into the (0,1) range, making it easy to interpret (closer to 1 → very well-resourced; closer to 0 → poorly-resourced).

In [ ]:
sql = text("""
ALTER TABLE public.sa2_summary
  ADD COLUMN IF NOT EXISTS z_business          DOUBLE PRECISION,
  ADD COLUMN IF NOT EXISTS z_stops             DOUBLE PRECISION,
  ADD COLUMN IF NOT EXISTS z_schools           DOUBLE PRECISION,
  ADD COLUMN IF NOT EXISTS z_poi               DOUBLE PRECISION,
  ADD COLUMN IF NOT EXISTS z_total             DOUBLE PRECISION,
  ADD COLUMN IF NOT EXISTS well_resourced_score DOUBLE PRECISION;

WITH stats AS (
  SELECT
    AVG(business_count)      AS mean_bc,
    STDDEV_POP(business_count) AS sd_bc,
    AVG(stops_count)         AS mean_sc,
    STDDEV_POP(stops_count)    AS sd_sc,
    AVG(catchments_count)    AS mean_cc,
    STDDEV_POP(catchments_count) AS sd_cc,
    AVG(poi_count)           AS mean_pc,
    STDDEV_POP(poi_count)      AS sd_pc
  FROM public.sa2_summary
)
UPDATE public.sa2_summary
SET
  z_business = (business_count - stats.mean_bc) / NULLIF(stats.sd_bc,0),
  z_stops    = (stops_count    - stats.mean_sc) / NULLIF(stats.sd_sc,0),
  z_schools  = (catchments_count - stats.mean_cc) / NULLIF(stats.sd_cc,0),
  z_poi      = (poi_count      - stats.mean_pc) / NULLIF(stats.sd_pc,0),
  z_total    = (
      (business_count   - stats.mean_bc) / NULLIF(stats.sd_bc,0)
    + (stops_count      - stats.mean_sc) / NULLIF(stats.sd_sc,0)
    + (catchments_count - stats.mean_cc) / NULLIF(stats.sd_cc,0)
    + (poi_count        - stats.mean_pc) / NULLIF(stats.sd_pc,0)
  ),
  well_resourced_score = 1.0 / (1.0 + EXP(-(
      (business_count   - stats.mean_bc) / NULLIF(stats.sd_bc,0)
    + (stops_count      - stats.mean_sc) / NULLIF(stats.sd_sc,0)
    + (catchments_count - stats.mean_cc) / NULLIF(stats.sd_cc,0)
    + (poi_count        - stats.mean_pc) / NULLIF(stats.sd_pc,0)
  )))
FROM stats;
""")

conn.execute(sql)

summary_df = query(
    conn,
    """
    SELECT 
      sa2_code, 
      sa2_name, 
      z_business, 
      z_stops, 
      z_schools, 
      z_poi, 
      z_total, 
      well_resourced_score
    FROM public.sa2_summary
    """
)
print(summary_df.head())

In [ ]:
import pandas as pd
from scipy.stats import pearsonr
from sqlalchemy import text

# 1) Load SA2 scores and incomes into a DataFrame
summary_df_raw = query(
    conn,
    """
    SELECT
      s.sa2_name,
      r.sa4_name_2021,
      s.well_resourced_score,
      i.median_income
    FROM sa2_summary AS s
    JOIN income       AS i USING (sa2_name)
    JOIN sa2_regions  AS r USING (sa2_name)
    """
)
print(f"Rows fetched from SQL: {len(summary_df_raw)}")

# Handle potential non-string median_income before .str.replace
summary_df = summary_df_raw.copy() # Work on a copy
summary_df["median_income"] = summary_df["median_income"].astype(str).str.replace(",", "")
summary_df["median_income"] = pd.to_numeric(
    summary_df["median_income"],
    errors="coerce"
)
print(f"Rows after median_income numeric conversion: {len(summary_df)}")
print(f"NaNs in median_income after coercion: {summary_df['median_income'].isna().sum()}")
print(f"NaNs in well_resourced_score: {summary_df['well_resourced_score'].isna().sum()}")

# 3) Overall Pearson correlation across all 67 SA2 regions
r_overall, p_overall = pearsonr(
    summary_df["well_resourced_score"],
    summary_df["median_income"]
)
print(f"Overall (n={len(summary_df)}): Pearson r = {r_overall:.3f}, p = {p_overall:.3f}")

# 4) Pearson correlation within each SA4 region
regions = ["Sydney - Blacktown", "Sydney - Parramatta", "Sydney - Ryde"]
for region in regions:
    sub = summary_df[summary_df["sa4_name_2021"] == region]
    r, p = pearsonr(sub["well_resourced_score"], sub["median_income"])
    print(f"{region} (n={len(sub)}): Pearson r = {r:.3f}, p = {p:.3f}")

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr, spearmanr

# 1) Load SA2 data with z-scores and incomes
sql = """
SELECT
  s.sa2_name,
  r.sa4_name_2021,
  s.well_resourced_score,
  s.z_business,
  s.z_stops,
  s.z_schools,
  s.z_poi,
  i.median_income
FROM sa2_summary AS s
JOIN income     AS i USING (sa2_name)
JOIN sa2_regions AS r USING (sa2_name)
"""
df = query(conn, sql).dropna()

# 2) Clean median_income and convert to float
df["median_income"] = df["median_income"].str.replace(",", "")
df["median_income"] = pd.to_numeric(df["median_income"], errors="coerce")
df = df.dropna(subset=["well_resourced_score", "median_income"])

# List of component columns and the composite
components    = ["z_business", "z_stops", "z_schools", "z_poi"]
composite_col = "well_resourced_score"

# 3.1 Direct influence: Pearson r of each component vs composite
print("1) Direct Pearson correlations with final score:")
for comp in components:
    r, p = pearsonr(df[comp], df[composite_col])
    print(f"   {comp:>12s} → r = {r:.3f}, p = {p:.3f}")

# 3.2 Leave-one-out rank stability: drop each component, re-rank, then Spearman rho
orig_rank = df[composite_col].rank(method="average")
print("\n2) Leave-one-out Spearman rank correlations:")
for comp in components:
    others   = [c for c in components if c != comp]
    raw_sum  = df[others].sum(axis=1)
    mod_score = 1 / (1 + np.exp(-raw_sum))  # reapply sigmoid if used originally
    mod_rank  = mod_score.rank(method="average")
    rho, p    = spearmanr(orig_rank, mod_rank)
    print(f"   without {comp:>8s} → ρ = {rho:.3f}, p = {p:.3f}")

# 3.3 Distribution shifts: compare mean, median, std of drop-one composites vs original
orig_stats = (
    df[composite_col].mean(),
    df[composite_col].median(),
    df[composite_col].std()
)
print("\n3) Distribution statistics comparison (mean, median, std):")
print(f"   original       : mean = {orig_stats[0]:.3f}, median = {orig_stats[1]:.3f}, std = {orig_stats[2]:.3f}")
for comp in components:
    others    = [c for c in components if c != comp]
    raw_sum   = df[others].sum(axis=1)
    mod_score = 1 / (1 + np.exp(-raw_sum))
    stats     = (mod_score.mean(), np.median(mod_score), mod_score.std())
    print(f"   without {comp:>8s} : mean = {stats[0]:.3f}, median = {stats[1]:.3f}, std = {stats[2]:.3f}")

### 1. Score Distribution
**Purpose:**  
Shows how well-resourced scores are distributed across all SA2 regions in your selected zones.

**Key Insights:**
- Reveals whether scores cluster around certain values (e.g., mostly average) or span the full range (0–1).
- The boxplot compares score distributions between your three SA4 zones, highlighting any systematic differences.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sqlalchemy import text

# Get the scores data with qualified column names
scores_df = pd.read_sql(text("""
    SELECT 
        public.sa2_summary.sa2_name,  -- Specify which table's sa2_name to use
        well_resourced_score, 
        sa4_name_2021 
    FROM public.sa2_summary 
    JOIN public.sa2_regions USING (sa2_code)
"""), conn)

# Set style
sns.set_style("whitegrid")
plt.figure(figsize=(10, 6))

# Histogram of scores
plt.subplot(1, 2, 1)
sns.histplot(scores_df['well_resourced_score'], bins=15, kde=True, color='skyblue')
plt.title('Distribution of Well-Resourced Scores')
plt.xlabel('Score')
plt.ylabel('Number of SA2 Regions')


# Boxplot by SA4 zone - updated to avoid deprecation warning
plt.subplot(1, 2, 2)
sns.boxplot(data=scores_df, x='sa4_name_2021', y='well_resourced_score', 
            hue='sa4_name_2021', palette='pastel')
plt.title('Score Distribution by SA4 Zone')
plt.xlabel('SA4 Zone')
plt.ylabel('Score')
plt.xticks(rotation=45, ha='right')

plt.tight_layout()
plt.show()

### 2. Choropleth Map
**Purpose:**  
Displays geographic patterns in well-resourced scores across your selected regions.

**Key Insights:**
- Identifies spatial clusters of high/low scores (e.g., city centers vs. suburbs).
- SA4 boundaries provide context for regional disparities.
- Color gradient (red–yellow–green) intuitively shows performance.


In [ ]:
#!pip install contextily

In [ ]:
import geopandas as gpd
import contextily as ctx

# Get the spatial data with scores
map_df = gpd.read_postgis(text("""
    SELECT 
        sr.sa2_code,
        sr.sa2_name,
        sr.geometry,
        ss.well_resourced_score,
        sr.sa4_name_2021
    FROM public.sa2_regions sr
    JOIN public.sa2_summary ss ON sr.sa2_code = ss.sa2_code
    WHERE sr.sa4_name_2021 IN ('Sydney - Blacktown', 'Sydney - Parramatta', 'Sydney - Ryde')
"""), conn, geom_col='geometry')

# Plot the map
fig, ax = plt.subplots(figsize=(12, 10))

# Plot the choropleth
map_df.plot(column='well_resourced_score', 
            ax=ax, 
            legend=True,
            legend_kwds={'label': "Well-Resourced Score", 'shrink': 0.5},
            cmap='RdYlGn',  # Red-Yellow-Green color scale
            edgecolor='black',
            linewidth=0.3,
            missing_kwds={'color': 'lightgrey'})

# Add SA4 boundaries for context
sa4_boundaries = gpd.read_postgis(text("""
    SELECT 
        sa4_name_2021,
        ST_Union(geometry) as geometry
    FROM public.sa2_regions
    WHERE sa4_name_2021 IN ('Sydney - Blacktown', 'Sydney - Parramatta', 'Sydney - Ryde')
    GROUP BY sa4_name_2021
"""), conn, geom_col='geometry')

sa4_boundaries.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=1.5)

# Add labels for SA4 zones
for idx, row in sa4_boundaries.iterrows():
    ax.annotate(text=row['sa4_name_2021'], 
                xy=row['geometry'].centroid.coords[0],
                ha='center', 
                fontsize=10,
                bbox=dict(boxstyle="round", facecolor='white', alpha=0.7))

# Add basemap
ctx.add_basemap(ax, crs=map_df.crs.to_string(), source=ctx.providers.CartoDB.Positron)

plt.title('Well-Resourced Scores by SA2 Region')
plt.axis('off')
plt.tight_layout()
plt.show()

### 3. Income Correlation
**Purpose:**  
Tests whether wealthier areas tend to be better resourced.

**Key Insights:**
- Positive correlation suggests resources align with income levels.
- Weak/no correlation implies your score captures independent factors.
- Outliers (high-score/low-income or vice versa) may reveal interesting exceptions.


In [ ]:
# Get data with income - fixed type mismatch and convert data types
corr_df = pd.read_sql(text("""
    SELECT 
        ss.sa2_name,
        ss.well_resourced_score,
        i.median_income,
        sr.sa4_name_2021
    FROM public.sa2_summary ss
    JOIN public.income i ON ss.sa2_code::text = i.sa2_code21::text
    JOIN public.sa2_regions sr ON ss.sa2_code = sr.sa2_code
"""), conn)

# Convert columns to numeric types
corr_df['well_resourced_score'] = pd.to_numeric(corr_df['well_resourced_score'], errors='coerce')
corr_df['median_income'] = pd.to_numeric(corr_df['median_income'], errors='coerce')

# Drop any rows with missing values after conversion
corr_df = corr_df.dropna(subset=['well_resourced_score', 'median_income'])

# Now create the plot with the properly formatted data
plt.figure(figsize=(10, 6))
sns.regplot(data=corr_df, 
            x='well_resourced_score', 
            y='median_income',
            scatter_kws={'alpha': 0.6},
            line_kws={'color': 'red'})

plt.title('Well-Resourced Score vs Median Income')
plt.xlabel('Well-Resourced Score')
plt.ylabel('Median Income ($)')
plt.grid(True)

# Calculate correlation coefficient
correlation = corr_df['well_resourced_score'].corr(corr_df['median_income'])
plt.annotate(f'Correlation: {correlation:.2f}', 
             xy=(0.05, 0.95), 
             xycoords='axes fraction',
             bbox=dict(boxstyle="round", facecolor='white', alpha=0.8))

plt.show()
correlation

### 4. Component Analysis
**Purpose:**  
Breaks down which factors (businesses, stops, schools, POIs) most influence scores.

**Key Insights:**
- Shows which components vary most (wide box = big differences between regions).
- Components above/below the zero line indicate strengths/weaknesses overall.
- Helps diagnose whether certain resources are consistently scarce.


In [ ]:
# Get the z-scores data
components_df = pd.read_sql(text("""
    SELECT 
        sa2_name,
        z_business,
        z_stops,
        z_schools,
        z_poi,
        well_resourced_score
    FROM public.sa2_summary
    ORDER BY well_resourced_score DESC
"""), conn)

# Melt the dataframe for easier plotting
melted_df = pd.melt(components_df, 
                    id_vars=['sa2_name', 'well_resourced_score'], 
                    value_vars=['z_business', 'z_stops', 'z_schools', 'z_poi'],
                    var_name='Component', 
                    value_name='Z-Score')

# Create more readable component names
component_names = {
    'z_business': 'Businesses',
    'z_stops': 'Transit Stops',
    'z_schools': 'Schools',
    'z_poi': 'Points of Interest'
}
melted_df['Component'] = melted_df['Component'].map(component_names)

# Plot the component contributions
plt.figure(figsize=(12, 6))
sns.boxplot(data=melted_df, x='Component', y='Z-Score', palette='Set2', hue='Component')
plt.title('Distribution of Z-Scores by Component', fontsize=14)
plt.xlabel('Component', fontsize=12)
plt.ylabel('Z-Score', fontsize=12)

# Add reference line at 0
plt.axhline(0, color='gray', linestyle='--', alpha=0.7, label='Zero Reference')

# Add a bit more context with grid
plt.grid(axis='y', linestyle='--', alpha=0.3)

# Add some statistics text
for i, comp in enumerate(component_names.values()):
    subset = melted_df[melted_df['Component'] == comp]['Z-Score']
    median = subset.median()
    plt.annotate(f'Median: {median:.2f}', 
                 xy=(i, subset.min() - 0.2), 
                 ha='center',
                 fontsize=9)

plt.tight_layout()
plt.show()

### 5. Top/Bottom Performers
**Purpose:**  
Highlights the highest and lowest scoring regions by name.

**Key Insights:**
- Reveals specific areas to examine as success stories or priorities for improvement.
- Large gaps between top and bottom indicate inequality in resource distribution.


In [ ]:
# Get top and bottom 5 regions
top_5 = components_df.nlargest(5, 'well_resourced_score')
bottom_5 = components_df.nsmallest(5, 'well_resourced_score')

# Add a category column to distinguish top from bottom
top_5['category'] = 'Top 5'
bottom_5['category'] = 'Bottom 5'
top_bottom_df = pd.concat([top_5, bottom_5])

# Sort the combined dataframe for better visual ordering
top_bottom_df = top_bottom_df.sort_values('well_resourced_score')

# Plot the scores with enhanced styling
plt.figure(figsize=(12, 6))
ax = sns.barplot(data=top_bottom_df, 
                x='well_resourced_score', 
                y='sa2_name',
                hue='category',  # Color bars based on top/bottom category
                palette={'Top 5': 'forestgreen', 'Bottom 5': 'firebrick'},
                dodge=False)  # Don't dodge the bars

# Add the score values as text at the end of each bar
for i, v in enumerate(top_bottom_df['well_resourced_score']):
    ax.text(v + 0.01, i, f'{v:.2f}', va='center')

plt.title('Top and Bottom 5 Performing SA2 Regions', fontsize=14)
plt.xlabel('Well-Resourced Score', fontsize=12)
plt.ylabel('SA2 Region', fontsize=12)
plt.xlim(0, max(top_bottom_df['well_resourced_score']) * 1.1)  # Dynamic limit with space for text
plt.grid(axis='x', linestyle='--', alpha=0.7)

# Add a legend in a good position
plt.legend(title='', loc='lower right')

plt.tight_layout()
plt.show()

### 6. Interactive Map (Optional)
**Purpose:**  
Allows readers to explore scores at their own pace.

**Key Features:**
- Hover tooltips show exact scores and region names.
- Zoom/pan functionality enables neighborhood-level inspection.
- Useful for digital reports or presentations.

In [ ]:
#!pip install folium

In [ ]:
import folium
from folium.plugins import MarkerCluster, FeatureGroupSubGroup
import numpy as np

# Create a base map centered on your regions
m = folium.Map(location=[-33.8, 151.0], zoom_start=11, tiles='cartodbpositron')

# Add multiple base tile layers
folium.TileLayer('cartodbpositron', name='Light Map').add_to(m)
folium.TileLayer('cartodbdark_matter', name='Dark Map').add_to(m)
folium.TileLayer('OpenStreetMap', name='OpenStreetMap').add_to(m)
folium.TileLayer('Stamen Terrain', name='Terrain').add_to(m)

# Create a custom colormap function
def get_color(score):
    if score is None:
        return '#CCCCCC'  # Gray for missing data
    elif score >= 0.8:
        return '#1a9850'  # Dark green
    elif score >= 0.6:
        return '#91cf60'  # Light green
    elif score >= 0.4:
        return '#ffffbf'  # Yellow
    elif score >= 0.2:
        return '#fc8d59'  # Orange
    else:
        return '#d73027'  # Red

# Style function for regions
def style_function(feature):
    score = feature['properties']['well_resourced_score']
    return {
        'fillColor': get_color(score),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7
    }

# Highlight function for hover effect
def highlight_function(feature):
    return {
        'weight': 3,
        'color': '#666',
        'dashArray': '',
        'fillOpacity': 0.9
    }

# Add choropleth layer with custom styling
choropleth = folium.GeoJson(
    map_df,
    name='Well-Resourced Scores',
    style_function=style_function,
    highlight_function=highlight_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['sa2_name', 'well_resourced_score', 'sa4_name_2021'],
        aliases=['Area:', 'Score:', 'Region:'],
        localize=True,
        sticky=True,
        labels=True,
        style="""
            background-color: #F0EFEF;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px 3px 3px rgba(0,0,0,0.4);
            font-size: 14px;
            padding: 10px;
        """
    )
).add_to(m)

# Add a legend
legend_html = '''
    <div style="position: fixed; 
        bottom: 50px; right: 50px; width: 180px; height: 170px; 
        border:2px solid grey; z-index:9999; font-size:14px;
        background-color: white; padding: 10px; border-radius: 5px">
    <p style="margin-top: 0; margin-bottom: 5px;"><b>Well-Resourced Score</b></p>
    <div style="display: flex; align-items: center; margin-bottom: 5px;">
        <span style="background-color:#1a9850; width:20px; height:20px; display:inline-block; margin-right:5px;"></span>
        <span>0.8 - 1.0</span>
    </div>
    <div style="display: flex; align-items: center; margin-bottom: 5px;">
        <span style="background-color:#91cf60; width:20px; height:20px; display:inline-block; margin-right:5px;"></span>
        <span>0.6 - 0.8</span>
    </div>
    <div style="display: flex; align-items: center; margin-bottom: 5px;">
        <span style="background-color:#ffffbf; width:20px; height:20px; display:inline-block; margin-right:5px;"></span>
        <span>0.4 - 0.6</span>
    </div>
    <div style="display: flex; align-items: center; margin-bottom: 5px;">
        <span style="background-color:#fc8d59; width:20px; height:20px; display:inline-block; margin-right:5px;"></span>
        <span>0.2 - 0.4</span>
    </div>
    <div style="display: flex; align-items: center; margin-bottom: 5px;">
        <span style="background-color:#d73027; width:20px; height:20px; display:inline-block; margin-right:5px;"></span>
        <span>0.0 - 0.2</span>
    </div>
    </div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Create a marker cluster group for top regions
marker_cluster = MarkerCluster(name="Top Regions").add_to(m)

# Add markers for top 5 regions
top_regions = map_df.nlargest(5, 'well_resourced_score')
for idx, row in top_regions.iterrows():
    # Get centroid coordinates
    centroid = row['geometry'].centroid
    popup_text = f"""
    <b>{row['sa2_name']}</b><br>
    Score: {row['well_resourced_score']:.2f}<br>
    Region: {row['sa4_name_2021']}
    """
    
    folium.Marker(
        location=[centroid.y, centroid.x],
        popup=folium.Popup(popup_text, max_width=300),
        icon=folium.Icon(color='green', icon='star', prefix='fa'),
        tooltip=f"Top Region: {row['sa2_name']}"
    ).add_to(marker_cluster)

# Add title
title_html = '''
    <div style="position: fixed; 
        top: 10px; left: 50%; transform: translateX(-50%);
        z-index:9999; font-size:18px; font-weight: bold; 
        background-color: white; padding: 10px; border-radius: 5px;
        border:2px solid grey; text-align: center;">
        Well-Resourced Scores by SA2 Region - Sydney
    </div>
'''
m.get_root().html.add_child(folium.Element(title_html))

# Add layer control
folium.LayerControl(collapsed=False).add_to(m)

# Add fullscreen control
from folium.plugins import Fullscreen
Fullscreen().add_to(m)

# Add measure tool
from folium.plugins import MeasureControl
m.add_child(MeasureControl(position='topleft', primary_length_unit='kilometers'))

# Display the map
m